# File and libraries

In [1]:
import pandas as pd
import numpy as np


file_tag = "Instacart Market Basket"

# DSLabs functions

In [2]:
%run "scripts/dslabs_functions.py"


# data functions

In [3]:
%run "scripts/data_functions.py"


data_functions lodaded


# sampling and testing

In [4]:

# test_data=True
test_data=False


# Define the sampling function
def sample_user_orders(data, fraction=0.1):
    # Get unique user_ids and order_ids
    unique_user_orders = data[['user_id', 'order_id']].drop_duplicates()
    sampled_user_orders = unique_user_orders.sample(frac=fraction)
    
    # Filter the dataset to include only the sampled users and orders
    sampled_data = data[data[['user_id', 'order_id']].apply(tuple, axis=1).isin(sampled_user_orders.apply(tuple, axis=1))]
    return sampled_data


# Load the data
orders = pd.read_csv('data/input/orders.csv')


if test_data==True:



    # Apply the sampling to each group 1%
    sample=0.05
    order_data = sample_user_orders(orders, fraction=sample)
    

else:


    order_data = orders
    # order_data = sample_user_orders(orders, fraction=sample)
    
    
# drop eval_set as its not necessary
order_data=order_data.drop(['eval_set'], axis=1)

print(order_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 6 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   order_number            int64  
 3   order_dow               int64  
 4   order_hour_of_day       int64  
 5   days_since_prior_order  float64
dtypes: float64(1), int64(5)
memory usage: 156.6 MB
None


## merge orders with prior

In [5]:
prior=pd.read_csv('data/input/order_products__prior.csv')


# merge the two dataframes on order id
data = pd.merge(prior,order_data , on='order_id', how='inner')

# class target column


In [6]:
target = "reordered"

values = data[target].value_counts(normalize=True) 
print(values)

reordered
1    0.589697
0    0.410303
Name: proportion, dtype: float64


In [7]:
data.shape

(32434489, 9)

In [8]:
summary5 = data.describe(include="all")

summary5

,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07,3.035642e+07
mean,1.710749e+06,2.557634e+04,8.351076e+00,5.896975e-01,1.029372e+05,1.714205e+01,2.738818e+00,1.342498e+01,1.110407e+01
std,9.873007e+05,1.409669e+04,7.126671e+00,4.918886e-01,5.946648e+04,1.753504e+01,2.090049e+00,4.246365e+00,8.778914e+00
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.559430e+05,1.353000e+04,3.000000e+00,0.000000e+00,5.142100e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00
50%,1.711048e+06,2.525600e+04,6.000000e+00,1.000000e+00,1.026110e+05,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00
75%,2.565514e+06,3.793500e+04,1.100000e+01,1.000000e+00,1.543910e+05,2.400000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,4.968800e+04,1.450000e+02,1.000000e+00,2.062090e+05,9.900000e+01,6.000000e+00,2.300000e+01,3.000000e+01


### additional date columns creation

In [9]:
# Categorize the time of day
def categorize_time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 22:
        return 'Evening'
    else:
        return 'Night'

data['order_time_of_day'] = data['order_hour_of_day'].apply(categorize_time_of_day)

data['is_weekend'] = data['order_dow'].apply(lambda x: 1 if x >= 5 else 0)  # 1 for weekend, 0 for weekday

data['is_peak_time_of_day']=data['order_time_of_day'].apply(lambda x: 1 if x in ['Morning','Afternoon'] else 0)

data['weeks_since_prior_order'] = data['days_since_prior_order'].apply(lambda x: round(x / 7, 0) if pd.notnull(x) else np.nan)



## encode date time to cyclic

In [10]:
from math import pi, sin, cos

# Function to apply sin and cos on an already-mapped cyclic feature
def apply_sin_cos_for_mapped_column(data, column):
    data[column + '_sin'] = np.sin(data[column])  # apply sine
    data[column + '_cos'] = np.cos(data[column])  # apply cosine
    return data

# Function to encode cyclic variables using sine and cosine
def cyclic_encode(value, x_max):
    # sine and cosine components to capture cyclic pattern
    value_sin = np.sin(2 * np.pi * value / x_max)
    value_cos = np.cos(2 * np.pi * value / x_max)
    return value_sin, value_cos


day_of_week_encoding_mapping={
    'Night':0,   
    'Morning':pi/2,
    'Afternoon':pi,
    'Evening':-pi/2,   
}

data['order_time_of_day_enc'] = encode_column_with_mapping(data, 'order_time_of_day', day_of_week_encoding_mapping)


# Encoding for local_hour (0-23)
data['order_hour_of_day_sin'], data['order_hour_of_day_cos'] = zip(*data['order_hour_of_day'].apply(lambda x: cyclic_encode(x, 23)))

# Encoding for day_of_week (0-6)
data['order_dow_sin'], data['order_dow_cos'] = zip(*data['order_dow'].apply(lambda x: cyclic_encode(x, 6)))


# user order dataframe for clustering

In [11]:
# Group by order_id and aggregate features
order_agg = data.groupby('order_id').agg({
    'add_to_cart_order': 'max',
    'reordered': 'mean',
}).rename(columns={'add_to_cart_order': 'num_products', 'reordered': 'reorder_rate'})


data=data.merge(order_agg, on='order_id', how='left')

In [12]:
# Drop product_id column and get unique values per order_id
distinct_orders = data.drop(columns=['product_id']).drop_duplicates()

# Group by user_id and aggregate features using named aggregations
user_agg = distinct_orders.groupby('user_id').agg(
    total_orders=('order_number', 'max'),  # Total number of orders
    mean_products=('num_products', 'mean'),  # mean products purchased
    max_products=('num_products', 'max'),  # max products ever purchased
    std_num_products=('num_products', 'std'),  # std products purchased
    mean_lag_between_orders=('days_since_prior_order', 'mean'),
    std_lag_between_orders=('days_since_prior_order', 'std'),
    mean_reorder_rate=('reorder_rate', 'mean'),
    mean_weekend_order_rate=('is_weekend', 'mean'),
    mean_order_dow=('order_dow', 'mean'),
    top_order_dow=('order_dow', lambda x: x.mode()[0]),
    order_dow_variety=('order_dow', lambda x: x.nunique()),
    top_order_hour=('order_hour_of_day', lambda x: x.mode()[0]),
    order_hour_variety=('order_hour_of_day', lambda x: x.nunique()),
    std_order_hour=('order_hour_of_day', 'std'),
    mean_peak_time_of_day_rate=('is_peak_time_of_day', 'mean'),
    top_order_time_of_day_enc=('order_time_of_day_enc', lambda x: x.mode()[0]),
    order_time_of_day_enc_variety=('order_time_of_day_enc', lambda x: x.nunique()),


).reset_index()

# Handle NaN values (if needed)
user_agg = user_agg.fillna(0)  # or user_agg.dropna()

user_agg.head()

,user_id,total_orders,mean_products,max_products,std_num_products,mean_lag_between_orders,std_lag_between_orders,mean_reorder_rate,mean_weekend_order_rate,mean_order_dow,top_order_dow,order_dow_variety,top_order_hour,order_hour_variety,std_order_hour,mean_peak_time_of_day_rate,top_order_time_of_day_enc,order_time_of_day_enc_variety
0,1,10,6.254237,9,1.582155,20.259259,9.304463,0.694915,0.000000,2.644068,4,4,7,7,3.500355,1.000000,1.570796,2
1,2,14,16.107692,26,5.469097,15.967033,9.119769,0.476923,0.030769,2.005128,2,5,9,5,1.649854,1.000000,1.570796,2
2,3,12,7.886364,11,2.042265,11.487179,4.869048,0.625000,0.000000,1.011364,0,4,16,6,1.454599,0.568182,3.141593,2
3,4,5,4.555556,7,2.120550,15.357143,8.580901,0.055556,0.500000,4.722222,4,3,15,3,1.745208,1.000000,3.141593,2
4,5,4,10.027027,12,2.315245,14.500000,4.263801,0.378378,0.000000,1.621622,3,3,18,3,2.588958,0.540541,3.141593,2


# enrich main order product dataframe

In [13]:


enriched_data = enrich_instacart_df(data)

            

# Final df to csv

In [14]:
if test_data==False:
    
    data.to_csv('data/instacart_pre_proc.csv',index=False)

    user_agg.to_csv('data/instacart_user_pre_proc.csv',index=False)

## sample df to csv

In [ ]:
# Apply the sampling to each group 20%
sample=0.10

In [ ]:
if test_data==False:
    
    sample_data = sample_user_orders(data, fraction=0.2)

    sample_data.to_csv('data/instacart_pre_proc_sample.csv',index=False)


In [19]:
if test_data==False:
    
    users_df_sample=user_agg.sample(frac=0.5)
    
    users_df_sample.to_csv('data/instacart_user_pre_proc_sample.csv',index=False)
